1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [7]:
def conversion(inp, format='dec'):
  """
  Converts a number in the format 'format', which can be
  'dec', 'bin' or 'hex' into the other possible two formats
  """
  if format == 'bin':
    out_d = int(bin(inp), 2)
    out_x = hex(out_d)
    print("The number in input is %s;" % bin(inp))
    print("Its decimal representation is %d, while its hex representation is %s" % (out_d, out_x))
    return out_d, out_x
  elif format == 'hex':
    out_d = int(hex(inp), 16)
    out_b = bin(out_d)
    print("The number in input is %s;" % hex(inp))
    print("Its decimal representation is %d, while its bin representation is %s" % (out_d, out_b))
    return out_d, out_b
  elif format == 'dec':
    out_b = bin(inp)
    out_x = hex(inp)
    print("The number in input is %d;" % inp)
    print("Its bin representation is %s, while its hex representation is %s" % (out_b, out_x))
    return out_b, out_x
  else:
    out_1 = None
    out_2 = None
    print("Format %s is not supported" % format)
    return out_1, out_2
  
d_to_b, d_to_x = conversion(41)
print("")
b_to_d, b_to_x = conversion(0b00101001,'bin')
print("")
h_to_d, h_to_b = conversion(0x29, 'hex')


The number in input is 41;
Its bin representation is 0b101001, while its hex representation is 0x29

The number in input is 0b101001;
Its decimal representation is 41, while its hex representation is 0x29

The number in input is 0x29;
Its decimal representation is 41, while its bin representation is 0b101001


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [8]:
def to_float(string):
  """
  Converts a string composed by zeros and ones into
  a float value according to IEEE 754 standard
  """
  bias = 127 # standard bias
  sign = int(string[0], 2) # sign extrapolation
  e = int(string[1:9], 2) # exponent extrapolation
  mantissa = 1
  for i in range(1,24):
    mantissa += int(string[i+8], 2)*2**(-i)
  result = (-1)**(sign)*mantissa*2**(e-bias)
  return result

word = '00000011111000000000000000000000' # "binary" string 32 bits long
float_word = to_float(word) # according to the lecture, should return 1.316554e-36
print("The float conversion of the input word resulted in: %.16e" % float_word)

The float conversion of the input word resulted in: 1.3165536729209620e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [9]:
under = over = temp = 1.0
counter_over = 0 # count number of subsequent doublings 
counter_under = 0 # count number of subsequent halvings

while over != float('inf'): # double the "over" value till it will equal 'inf'
  temp = over
  over *= 2
  counter_over += 1
print("Overflow occurred after %d doublings! Max limit of the system (up to a factor of 2) is: %.16e" % (counter_over, temp))

while under != 0: # halve the "under" value till  it will equal zero
  temp = under
  under /= 2
  counter_under += 1
print("Underflow occurred after %d halvings! Min limit of the system (up to a factor of 2) is: %.16e" % (counter_under, temp))

Overflow occurred after 1024 doublings! Max limit of the system (up to a factor of 2) is: 8.9884656743115795e+307
Underflow occurred after 1075 halvings! Min limit of the system (up to a factor of 2) is: 4.9406564584124654e-324


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [10]:
result = temp = a = 2
bits = 0
while True: # continue until you sum a quantity that makes the updated value not
            # pass an equal check wrt the previous value
  a /= 2
  temp += a
  if result != temp:
    result = temp
    bits += 1
  else:
    print("The machine (double) precision of this system corresponds to %.15e, using %d bits for the mantissa" % (a, bits))
    break


The machine (double) precision of this system corresponds to 1.110223024625157e-16, using 53 bits for the mantissa


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [22]:
#(a)
def quad_roots(a, b, c):
  sqr_delta = (b**2 - 4*a*c)**0.5
  p_root = (-b + sqr_delta)/(2*a) # sqr_delta is circa b, we encounter a cancelation 
  n_root = (-b - sqr_delta)/(2*a) # this is okay
  return n_root, p_root

a = c = 0.001
b = 1000
x1, x2 = quad_roots(a, b, c)
print("The results of (a) are -> x1 = %.12e, x2 = %.12e" % (x1, x2))
# we can guess that the problem is due to the orders of magnitude of difference between b^2 and 4ac

#(b)
def alt_quad_roots(a, b, c):
  sqr_delta = (b**2 - 4*a*c)**0.5
  p_num = -b + sqr_delta
  n_num = -b - sqr_delta

  p_root = 2*c/n_num # this is okay
  n_root = 2*c/p_num  # again, sqr_delta is too close to b and we encounter a cancelation
  return n_root, p_root

x1, x2 = alt_quad_roots(a, b, c)
print("The results of (b) are -> x1 = %.12e, x2 = %.12e" % (x1, x2))

#(c)
# we combine the results from the previous points to provide a stable method
def better_quad_roots(a, b, c):
  sqr_delta = (b**2 - 4*a*c)**0.5
  n_num = -b - sqr_delta

  p_root = 2*c/n_num # this is okay
  n_root = (-b - sqr_delta)/(2*a) # this is okay
  return n_root, p_root

x1, x2 = better_quad_roots(a, b, c)
print("The results of (c) are -> x1 = %.12e, x2 = %.12e" % (x1, x2))

The results of (a) are -> x1 = -9.999999999990e+05, x2 = -9.999894245993e-07
The results of (b) are -> x1 = -1.000010575513e+06, x2 = -1.000000000001e-06
The results of (c) are -> x1 = -9.999999999990e+05, x2 = -1.000000000001e-06


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [37]:
#(a)
def my_fun(x):
  return x*(x-1)

def derivative(x, delta):
  """
  Computes the derivative of f in x
  """
  return (my_fun(x+delta) - my_fun(x))/delta

x = 1 
delta = 10**(-2)
print("The derivative of f(x) with x = %f and delta = %.20e is: %f" % (x, delta, derivative(x, delta)))
# ...but analitically the result is different: check the markdown cell below

The derivative of f(x) with x = 1.000000 and delta = 1.00000000000000002082e-02 is: 1.010000


In this case we can see that the result is slightly different from the sharp value of 1 expected analitically. This happens because of the limited precision of the machine, and by printing out the value of $\delta$ with a not negligible number of significant figures we can detect that it is not perfectly equal to $10^{−2}$. Increasing the absolute value of the negative exponent for $\delta$, we get the best result (i.e. the one that approaches better the analitical value) for $\delta = 10^{−8}$, and increasing it further will lead to worse outcomes.

In [38]:
#(b)
delta_list = [10**(-2*k) for k in range(2,8)]

for delta in delta_list:
  print("The derivative of f(x) with x = %f and delta = %.20e is: %.16e" % (x, delta, derivative(x, delta)))

The derivative of f(x) with x = 1.000000 and delta = 1.00000000000000004792e-04 is: 1.0000999999998899e+00
The derivative of f(x) with x = 1.000000 and delta = 9.99999999999999954748e-07 is: 1.0000009999177333e+00
The derivative of f(x) with x = 1.000000 and delta = 1.00000000000000002092e-08 is: 1.0000000039225287e+00
The derivative of f(x) with x = 1.000000 and delta = 1.00000000000000003643e-10 is: 1.0000000828403710e+00
The derivative of f(x) with x = 1.000000 and delta = 9.99999999999999979887e-13 is: 1.0000889005833413e+00
The derivative of f(x) with x = 1.000000 and delta = 9.99999999999999998819e-15 is: 9.9920072216265088e-01


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [43]:
#(a)
semicircle = lambda x: (1-x**2)**0.5

def integral(func, N, lower_ext=-1, upper_ext=1):
  """
  Compute the integral (Riemann definition) of function func
  in interval (lower_ext, upper_ext) with N slices
  """
  h = (upper_ext - lower_ext)/N
  I = 0
  for k in range (1, N):
    I += func(lower_ext + h*k)*h
  return I

N = 100
I = integral(semicircle, N)

print("Integral of semicircle with radius 1, Riemann approximation with N = %d: %f" % (N, I))
# we can see that the result is not that bad but neither acceptable if we want some precision;
# N has to be increased to achieve that

Integral of semicircle with radius 1, Riemann approximation with N = 100: 1.569134


In [70]:
#(b)
import time
import math

N = 23*10**5 # by iterative searching, make a guess that is close to make the runtime sligthly less than 1s
runtime = 0
max_time = 1 # seconds
while runtime < max_time:
  best_N = N
  best_time = runtime
  start = time.time()
  I_1 = integral(semicircle, N)
  runtime = time.time() - start
  N += 10**3

error_1 = abs(I_1 - math.pi/2)
print("With N=%d -> I=%.8f, error=%.3e and runtime=%.5f seconds" % (best_N, I_1, error_1, best_time))

N = 10**8 # by iterative searching, make a guess that is close to make the runtime sligthly less than 60s
max_time = 60 # seconds
while runtime < max_time:
  best_N = N
  best_time = runtime
  start = time.time()
  I_60 = integral(semicircle, N)
  runtime = time.time() - start
  N = int(N*1.5)

error_60 = abs(I_60 - math.pi/2)
print("With N=%d -> I=%.8f, error=%.3e  and runtime=%.5f seconds" % (best_N, I_60, error_60, best_time))

# the gain is of circa 2 orders of magnitude if we run the program for slighly less than 60s


With N=2380000 -> I=1.57079633, error=4.529e-10 and runtime=0.94389 seconds
With N=225000000 -> I=1.57079633, error=1.238e-12  and runtime=59.49574 seconds
